### Basic imports

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os

### 1. Read in Data from csv --> save the master_loan_tape csv from INTRALINKS into the `raw_data` folder to run

In [ ]:
if not os.listdir('raw_data/'):
    print('Download master_loan_tape.csv from Intralinks and save it in the raw_data folder')

else:
    loan_data = pd.read_csv("raw_data/master_loan_tape.csv")
    # format date columns to datetime data types
    date_cols = [c for c in loan_data.columns if str(c)[-2:]=='Dt']
    for col in date_cols:
        loan_data[col] = pd.to_datetime(loan_data[col])

    # Take a look at the top of the dataframe
    loan_data.head()

In [ ]:
# Slice the data frame to only 21+ year maturity loans
data_slice = loan_data[loan_data['MatBucket']=='21+']

### 2. Use pool.py to organize cohorts --> loans are split into yyyy.mm cohorts

In [ ]:
from pooler import pool

# Define a function to create static pools of Loans from the DataFrame
def create_pooler(in_df:pd.DataFrame)-> pool.Pooler:
    temp = in_df.set_index('GP')
    temp = temp.to_dict()
    loans_dict = {}
    for gp in temp['NoteDt'].keys():
        loans_dict[str(gp)] = pool.Loan(gp, pd.to_datetime(temp['NoteDt'][gp]))
        loans_dict[str(gp)].maturity_dt = temp['MaturityDt'][gp]
        loans_dict[str(gp)].maturity_mths_qty = temp['MaturityMthsQty'][gp]
        loans_dict[str(gp)].default_dt = temp['DefaultDt'][gp]
        loans_dict[str(gp)].default_mths_qty = temp['DefaultMthsQty'][gp]
        loans_dict[str(gp)].prepay_dt = temp['PrepayDt'][gp]
        loans_dict[str(gp)].prepay_mths_qty = temp['PrepayMthsQty'][gp]

    return pool.Pooler(loans_dict)

my_pooler = create_pooler(data_slice)
my_pooler.build_triangles_counts()

pool_dict = {}
for k, v in my_pooler.triangles.items():
    pool_dict[k] = dict(outstanding=v[0], prepayments=v[1], defaults=v[2])


### 3. Convert Pool dictionaries into Dataframe

In [ ]:
# Create Pool Dataframe
df_pool = pd.DataFrame.from_dict( pool_dict, orient='index')
df_pool.index = [float(e) for e in df_pool.index.to_list()]
df_pool = df_pool.sort_index()
# Show the dataframe here
df_pool.head()

In [ ]:
# Format array lengths
max_row_length = df_pool.shape[0]
count = -1
for i, row in df_pool.iterrows():
    count+=1
    for col in df_pool.columns:
        # ----------------------------------------------------------------------------------------------
        #Method 1: This line will simply truncate the array to the right length:
        # ----------------------------------------------------------------------------------------------
        # df_pool.at[i,col] = row[col][:(max_row_length-count)]
        # ----------------------------------------------------------------------------------------------
        #Method 2: Pad arrays with NaNs --> this will truncate the array then fill it back in with NaNs
        # ----------------------------------------------------------------------------------------------
        arr = row[col][:(max_row_length-count)].astype(float)
        padded_arr = np.pad(arr, (0, max_row_length - (max_row_length-count) ), mode='constant', constant_values=np.nan)
        df_pool.at[i,col] = padded_arr


### 4. Group into Annual cohorts and calculate SMM and CPR

In [ ]:
# This is where we start abstracting away from the Data --> you're now looking at Annual Cohorts grouped together
df_pool['Year'] = df_pool.index.astype(int)
year_grouped = df_pool.groupby('Year')
year_grouped = year_grouped.agg(np.nansum)
year_grouped['smm'] = (year_grouped['prepayments']+year_grouped['defaults'])/year_grouped['outstanding']
year_grouped['cpr'] = (1-(1-year_grouped['smm'])**12)

### Reshape the Yearly cohorts data from Months on Book to Year on Book  
`[i,...........,n=MoB] ---> applymap(aggregate_method)`  
 
`[[i,...]`  
`[i,...]`  
`[i,...]`  
`[i,...]`  
`[i,...]...n_years=YoB]`

In [ ]:
def aggregate_annual_sums(in_arr):
    n_years = len(in_arr) // 12
    arr_2d = in_arr[:n_years*12].reshape(n_years,12)
    return np.nansum(arr_2d,axis=1)

def aggregate_annual_averages(in_arr):
    n_years = len(in_arr) // 12
    arr_2d = in_arr[:n_years*12].reshape(n_years,12)
    return np.nanmean(arr_2d,axis=1)

arr = year_grouped[['cpr']]
cpr_heat = arr.applymap(aggregate_annual_averages).to_dict()
cpr_heat = cpr_heat['cpr']


cpr_heat = pd.DataFrame.from_dict(cpr_heat, orient='index')

### 5. Get Line Plots (or just the data for line plots)

In [ ]:
# First attempt --> 
cumsum = np.nancumsum(cpr_heat, axis=1)
lifetime_avg= pd.DataFrame(cumsum/np.arange(1,cpr_heat.shape[1]+1), columns=cpr_heat.columns, index=cpr_heat.index)

In [ ]:
def create_lifetime_averages(cpr_df:pd.DataFrame)-> pd.DataFrame:
    # get the cumulative sum of each row WHILE ignoring NaN values (otherwise the denominator is off)
    cumulative_sum = np.nancumsum(cpr_df.values, axis=1)
    # compute the number of non-NaN values in each row
    num_non_nan = (~np.isnan(cpr_df.values)).cumsum(axis=1)
    # get ROW-WISE average up until the first NaN value is encountered
    row_avg = np.where(np.isnan(cpr_df), np.nan, cumulative_sum / num_non_nan)
    # create new dataframe with row-wise averages
    lifetime_df = pd.DataFrame(row_avg, columns=cpr_df.columns, index=cpr_df.index)
    return lifetime_df

lifetime_cprs = create_lifetime_averages(cpr_heat)

lifetime_cprs.fillna('')


In [ ]:
import plotly.express as px

fig = px.line(lifetime_cprs.transpose(), line_shape= 'spline', title="Lifetime Average CPR by Year from Origination", markers=True)
fig.update_layout(yaxis_title='CPR', xaxis_title= 'Year from Origination', yaxis=dict(tickformat='0.0%'))